# Make wide k-mer count dataframe

Reads in csvs of FracMinHash sketches and converts them into a wide dataframe. This should probably be converted to a snakemake script and be added to the workflow, but will be a notebook for now.

Note I know the second one fails from trying to do too many joins with too little RAM. The subsequent notebooks just use PA14, which is a smaller compendium.

In [1]:
setwd("..")

In [3]:
library(readr)
library(dplyr)
library(tibble)
library(tidyr)
library(purrr)
library(ggplot2)
library(ggpubr)
library(Seurat)

## PA14

In [5]:
# read in metadata
metadata <- read_tsv("inputs/metadata.tsv", show_col_types = F)
metadata_pa14 <- metadata %>%
  dplyr::filter(strain_type == "PA14")

In [6]:
# read in k-mer abundances
pa14_files <- paste0("outputs/srx_sourmash_sketch_filtered_csv/", metadata_pa14$experiment, "_k21.csv")
raw_pa14_kmers_long <- pa14_files %>%
  purrr::set_names() %>%
  purrr::map_dfr(read_csv, skip = 1, col_names = c("minhash", "abund"), show_col_types = F, .id = "sample") %>%  # read in sketches
  dplyr::mutate(sample = gsub("_k21\\.csv", "", basename(sample)))

In [8]:
# pivot from long to wide; shares minhash index
raw_pa14_kmers <- raw_pa14_kmers_long %>%
  pivot_wider(id_cols = minhash, names_from = sample, values_from = abund) %>%
  replace(is.na(.), 0) # replace NAs with zero abund

In [9]:
write_csv(raw_pa14_kmers, "outputs/srx_sourmash_sketch_wide_csv/pa14_k21.csv")

## PAO1

In [10]:
metadata_pao1 <- metadata %>%
  dplyr::filter(strain_type == "PAO1")

In [11]:
# read in k-mer abundances
pao1_files <- paste0("outputs/srx_sourmash_sketch_filtered_csv/", metadata_pao1$experiment, "_k21.csv")
raw_pao1_kmers_long <- pao1_files %>%
  purrr::set_names() %>%
  purrr::map_dfr(read_csv, skip = 1, col_names = c("minhash", "abund"), show_col_types = F, .id = "sample") %>%  # read in sketches
  dplyr::mutate(sample = gsub("_k21\\.csv", "", basename(sample)))

In [12]:
# pivot from long to wide; shares minhash index
raw_pao1_kmers <- raw_pao1_kmers_long %>%
  pivot_wider(id_cols = minhash, names_from = sample, values_from = abund) %>%
  replace(is.na(.), 0) # replace NAs with zero abund

Warning message in nrow * ncol:
“NAs produced by integer overflow”


ERROR: [1m[33mError[39m:[22m
[33m![39m Internal error in `compact_rep()`: Negative `n` in `compact_rep()`.


In [ ]:
write_csv(raw_pao1_kmers, "outputs/srx_sourmash_sketch_wide_csv/pao1_k21.csv")